In [16]:
from dotenv import load_dotenv
import os
import time

from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

In [17]:
load_dotenv(dotenv_path = r'C:\Users\User\Desktop\Project LLMs\RAG-LangChain\data\.env')
nvidia_api_key = os.getenv('NVIDIA_API_KEY')
typhoon_ai_api_key = os.getenv('TYPHOON_AI_API_KEY')
langsmith_api_key = os.getenv('LANGSMITH_API_KEY')
cohere_api_key = os.getenv('COHERE_API_KEY')

In [18]:
os.environ['LANGCHAIN_API_KEY'] = langsmith_api_key
os.environ['LANGSMITH_TRACING'] = 'true'

In [19]:
with open(r'C:\Users\User\Desktop\Project LLMs\RAG-LangChain\Evaluation\test_question.txt', 'r', encoding="utf-8") as file:
    test_questions = file.readlines()

test_questions = [question.strip() for question in test_questions]
len(test_questions)

40

In [20]:
cohere_embedding_model = CohereEmbeddings(model="embed-multilingual-v3.0",
                                           cohere_api_key = cohere_api_key)
bge_m3_embedding_model = OllamaEmbeddings(model="bge-m3:latest")

In [21]:
cohere_vectorstore = FAISS.load_local(r"C:\Users\User\Desktop\Project LLMs\RAG-LangChain\data\vectorstore\faiss_index_cohere", cohere_embedding_model, allow_dangerous_deserialization = True)
bge_m3_vectorstore = FAISS.load_local(r"C:\Users\User\Desktop\Project LLMs\RAG-LangChain\data\vectorstore\faiss_index_bge_m3", bge_m3_embedding_model, allow_dangerous_deserialization = True)

In [22]:
nvidiallama33_70 = ChatNVIDIA(model="meta/llama-3.3-70b-instruct",
                                api_key = nvidia_api_key)

openaityphoon2_70 = ChatOpenAI(base_url = 'https://api.opentyphoon.ai/v1',
                        model = 'typhoon-v2-70b-instruct',
                        api_key = typhoon_ai_api_key)

In [23]:
vectorstores = {'cohere': cohere_vectorstore,
                'bge_m3': bge_m3_vectorstore}

llms = {'llama33_70': nvidiallama33_70,
        'typhoon2_70': openaityphoon2_70}

projects = ['Cohere - LLaMA3.3 70B', 'Cohere - TYPHOON AI 2 70B', 'BGE M3 - LLaMA3.3 70B', 'BGE M3 - TYPHOON AI 2 70B']

In [24]:
def build_retriever_chain(model, vectorstore):
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are Junior, a friendly and knowledgeable male AI assistant.\nYour role is to assist users by providing academic advice, explanations, and helpful insights based on the given context.\nAlways ensure your responses are clear, concise, and easy to understand.\nUse only the provided context to answer questions, and do not rely on external knowledge.\nIf the context lacks sufficient information, politely ask the user for clarification.\nContext: {context}"),
        ('human', "{input}")
    ])

    chain = create_stuff_documents_chain(
        llm = model,
        prompt = prompt
    )

    retriever = vectorstore.as_retriever()

    retriever_chain = create_retrieval_chain(
        retriever = retriever,
        combine_docs_chain = chain
    )

    return retriever_chain


In [25]:
samples = test_questions[:2]

In [26]:
project_index = 0
for vector_name, vectorstore in vectorstores.items():
    print(f'Vector Name: {vector_name}')

    for model_name, llm in llms.items():
        current_project = projects[project_index]

        print(f'Model Name: {model_name}')
        print(f'Setting project to: {current_project}')

        minute_start_time = time.time()
        request_count_per_minute = 0
        
        LANGSMITH_TRACING= True
        LANGSMITH_ENDPOINT= "https://api.smith.langchain.com"
        LANGSMITH_API_KEY= langsmith_api_key

        os.environ['LANGSMITH_PROJECT'] = current_project
        
        
        # Create the chain with the updated environment
        retriever_chain = build_retriever_chain(llm, vectorstore)
        
        # Process questions...
        for i, question in enumerate(samples):


            if model_name == 'typhoon2_70':  
                time.sleep(0.2)

                # จัดการกับข้อจำกัด 50 requests/minute
                request_count_per_minute += 1
                if request_count_per_minute >= 45:  # ตั้งไว้ที่ 45 เพื่อเผื่อความปลอดภัย
                    elapsed_time = time.time() - minute_start_time
                    if elapsed_time < 60:
                        # ถ้ายังไม่ถึง 1 นาที แต่จำนวน request เกือบถึงขีดจำกัดแล้ว
                        time_to_wait = 60 - elapsed_time
                        print(f"Approaching minute limit. Waiting {time_to_wait:.2f} seconds...")
                        time.sleep(time_to_wait)
                    
                    # รีเซ็ตตัวนับเมื่อเริ่มนาทีใหม่
                    minute_start_time = time.time()
                    request_count_per_minute = 0
            
            try:
                print(f'Processing Question {i+1}...')
                results = retriever_chain.invoke({'input': question})
                print(f'Question-Answering Test {i+1}:')
                print(f'  Question: {results["input"]}')
                print(f'  Answer: {results["answer"]}')
            except Exception as e:
                print(f'Error on question {i+1}: {str(e)}')
                if 'rate limit' in str(e).lower():
                    wait_time = 15  # รอนานขึ้นถ้าเจอ rate limit error
                    print(f"Rate limit hit. Waiting {wait_time} seconds...")
                    time.sleep(wait_time)
                    # ลองอีกครั้ง
                    try:
                        results = retriever_chain.invoke({'input': question})
                        print(f'Question-Answering Test {i+1} (retry):')
                        print(f'  Question: {results["input"]}')
                        print(f'  Answer: {results["answer"]}')
                    except Exception as retry_error:
                        print(f'Retry failed: {str(retry_error)}')
            
            print('-----------------------------------------------')

        project_index += 1
        print(f'Finished project: {current_project}')
        print('\\n#==================================#\\n')

    print('\\n#==================================#\\n')

Vector Name: cohere
Model Name: llama33_70
Setting project to: Cohere - LLaMA3.3 70B
Processing Question 1...
Question-Answering Test 1:
  Question: วิชาที่เปิดสอนสำหรับภาควิชาเคมีชั้่นปีที่ 1 ปีการศึกษา 2567 เทอม 1 มาสัก 1 วิชา
  Answer: วิชาเคมีทั่วไป 1 หรือ GENERAL CHEMISTRY I รหัสวิชา 513101-2560 และ 513101-165 มีรายละเอียดการเรียนการสอดังนี้

- ปริมาณสัมพันธ์ 
- โครงสร้างอะตอมและสมบัติของธาตุตามตารางธาตุ 
- พันธะเคมี 
- แก๊ส 
- ของแข็ง 
- เทอร์โมไดนามิกส์ หรือ อุณหพลศาสตร์ 

มี 2 กลุ่ม โดยกลุ่มที่ 1 เปิดสอนให้กับนักศึกษาสาขาชีวภาพ แวดล้อม กระบวนการ เคมี ปี1-3 และกลุ่มที่ 2 เปิดสอนให้กับนักศึกษาสาขาชีวะ จุล ฟิสิกส์ ปิโตร อาหาร ปี1-3 และมี 2 คาบเรียน คือ วันพุธและวันศุกร์
-----------------------------------------------
Processing Question 2...
Question-Answering Test 2:
  Question: ช่วยแนะนำวิชาสำหรับภาควิชาคณิตศาสตร์หน่อยสัก 1 วิชา
  Answer: สำหรับภาควิชาคณิตศาสตร์แนะนำวิชา "เรื่องคัดเฉพาะทางคณิตศาสตร์ 1" รหัสวิชา 511481-2560 หน่วยกิต 3 ซึ่งเป็นวิชาบังคับของสาขาคณิตปี4-8 โดยมีรายละ